In [3]:
import csv
with open('dataset/MTA_Turnstile_2014_2.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    row1 = next(reader)
    print row1

{'REGULAR': 'REGULAR', '10/11/2014': '10/11/2014', 'R1': 'R1', '01:00:00': '05:00:00', '00-00-00': '00-00-00', '000114108': '000114114', 'R001': 'R001', 'BMT': 'BMT', 'A060': 'A060', '0000805439': '0000805459', 'WHITEHALL ST': 'WHITEHALL ST'}


In [27]:
import pandas as pd

data = pd.read_csv("dataset/MTA_Turnstile_2014_2.csv", sep=',', header=None, usecols=[1,6,7,9,10])
data.columns = ["Remote", "Date", "Hour", "Entry", "Exit"]
data.shape


(1868519, 5)

In [59]:
remote_data = pd.read_csv("dataset/Remote-Booth-Station.csv", sep=',', usecols=[0,2])
remote_data = remote_data.drop_duplicates()
remote_data = remote_data.drop_duplicates(['Remote'], keep='first')
print remote_data.shape
print remote_data.head()

(480, 2)
  Remote         Station
0   R001    WHITEHALL ST
3   R002       FULTON ST
6   R003   CYPRESS HILLS
7   R004    ELDERTS LANE
8   R005  FOREST PARKWAY


In [60]:
merged = data.merge(remote_data,how='inner',on='Remote')
print merged.shape
print merged.tail(10)

(1868519, 6)
        Remote        Date      Hour  Entry  Exit         Station
1868509   R552  12/25/2014  07:07:17  31889  2362  JOURNAL SQUARE
1868510   R552  12/25/2014  11:19:17  31890  2362  JOURNAL SQUARE
1868511   R552  12/25/2014  15:31:17  31898  2363  JOURNAL SQUARE
1868512   R552  12/25/2014  19:43:17  31905  2364  JOURNAL SQUARE
1868513   R552  12/25/2014  23:55:17  31910  2364  JOURNAL SQUARE
1868514   R552  12/26/2014  04:07:17  31910  2364  JOURNAL SQUARE
1868515   R552  12/26/2014  08:19:17  31918  2365  JOURNAL SQUARE
1868516   R552  12/26/2014  12:31:17  31956  2366  JOURNAL SQUARE
1868517   R552  12/26/2014  16:43:17  31972  2367  JOURNAL SQUARE
1868518   R552  12/26/2014  20:55:17  31986  2368  JOURNAL SQUARE


In [61]:
weather_data = pd.read_csv("dataset/clean-weather-data/weather.csv", sep=',', header=None)
weather_data.columns = ['Date','Event']
weather_data = weather_data.fillna('Normal')
weather_data.head()
print weather_data.shape

(2223, 2)


In [62]:
remerged = merged.merge(weather_data,how='inner',on='Date')
remerged[500000:510000]
remerged.shape

(1868519, 7)